In [1]:
import pandas as pd

# 📄 Absolute path to the CSV file
csv_path = r"C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\data\sensor\points\2025-04-15_17-43-22_points_694020.csv"

# 📥 Load the data
df = pd.read_csv(csv_path)

# 🖨 Check
print(f"✅ Points loaded: {len(df)}")
df.head()


✅ Points loaded: 694020


,X,Y,Z
0,1885.72,323.853,-52.5201
1,1885.71,323.912,-52.5371
2,1885.74,323.992,-52.4593
3,1885.72,324.037,-52.5426
4,1885.72,324.096,-52.5586


In [2]:
# STEP 2: Outlier removal by Z (using the 90% range)
z_low = df['Z'].quantile(0.05)
z_high = df['Z'].quantile(0.99)

print(f'🔍 Z range for filtering: {z_low:.2f} — {z_high:.2f}')

# Apply filtering
df_clean_z = df[(df['Z'] >= z_low) & (df['Z'] <= z_high)].copy()
print(f'✅ Points remaining after Z filtering: {len(df_clean_z)}')


🔍 Z range for filtering: -42.35 — -31.16
✅ Points remaining after Z filtering: 652386


In [3]:
# STEP 3: ROI filtering by Y and X (central 75%)

x_min, x_max = df_clean_z['X'].quantile([0.3, 0.9])
y_min, y_max = df_clean_z['Y'].quantile([0.367, 0.495])
z_min, z_max = df_clean_z['Z'].quantile([0.6, 0.999])

print(f'🔍 ROI Z range: {z_min:.2f} — {z_max:.2f}')
print(f'🔍 ROI X range: {x_min:.2f} — {x_max:.2f}')
print(f'🔍 ROI Y range: {y_min:.2f} — {y_max:.2f}')

df_clean_roi = df_clean_z[
    (df_clean_z['X'] >= x_min) & (df_clean_z['X'] <= x_max) &
    (df_clean_z['Y'] >= y_min) & (df_clean_z['Y'] <= y_max) &
    (df_clean_z['Z'] >= z_min) & (df_clean_z['Z'] <= z_max)
].copy()

print(f'✅ Points remaining after ROI filtering: {len(df_clean_roi)}')


🔍 ROI Z range: -41.67 — -31.21
🔍 ROI X range: 1819.05 — 1874.29
🔍 ROI Y range: 348.98 — 356.71
✅ Points remaining after ROI filtering: 22965


In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# 📥 Assumes that df_clean_roi is already created
df_input = df_clean_roi.copy()
n_clusters_local = 7

# 📌 Clustering along the Z-axis
z_vals = df_input[['Z']].values
kmeans = KMeans(n_clusters=n_clusters_local, random_state=42)
df_input['layer_id_2MethodKmean7'] = kmeans.fit_predict(z_vals)

# 🧭 Renumbering layers from bottom to top
centers = kmeans.cluster_centers_.flatten()
sorted_idx = np.argsort(centers)
mapping = {old: new for new, old in enumerate(sorted_idx)}
df_input['layer_id_2MethodKmean7'] = df_input['layer_id_2MethodKmean7'].map(mapping)

# 🧮 Statistics by layer
print("📊 Number of points in each layer:")
for lid in sorted(df_input['layer_id_2MethodKmean7'].unique()):
    count = (df_input['layer_id_2MethodKmean7'] == lid).sum()
    print(f"  - Layer {lid}: {count} points")

print(f"\n✅ Total number of points: {len(df_input)}")


📊 Number of points in each layer:
  - Layer 0: 1653 points
  - Layer 1: 3824 points
  - Layer 2: 3804 points
  - Layer 3: 3933 points
  - Layer 4: 3337 points
  - Layer 5: 3775 points
  - Layer 6: 2639 points

✅ Total number of points: 22965


In [5]:
import plotly.graph_objs as go

# Unique global layers
unique_layers = sorted(df_input['layer_id_2MethodKmean7'].unique())  # global_layer_id

# 🎯 Points for each layer
data = []
for lid in unique_layers:
    subset = df_input[df_input['layer_id_2MethodKmean7'] == lid]
    trace = go.Scatter3d(
        x=subset['X'],
        y=subset['Y'],
        z=subset['Z'],
        mode='markers',
        marker=dict(size=2),
        name=f'Layer {lid}',
        visible=True if lid == 0 else False  # Show only the first layer by default
    )
    data.append(trace)

# 🔘 Buttons to switch layers
buttons = []
for i, lid in enumerate(unique_layers):
    visible = [j == i for j in range(len(unique_layers))]
    buttons.append(dict(
        label=f"Layer {lid}",
        method="update",
        args=[{"visible": visible}, {"title": f"3D: Only Layer {lid}"}]
    ))

# Button to show all layers
buttons.insert(0, dict(
    label="All Layers",
    method="update",
    args=[{"visible": [True]*len(unique_layers)}, {"title": "3D: All Layers"}]
))

# ⚙️ Final chart assembly
fig = go.Figure(data=data)
fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        showactive=True,
        x=1.05, y=1,
        buttons=buttons
    )],
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title="3D: Only Layer 0 (default)"
)

# 👁 Show
# fig.show()
pass


In [6]:
import os

# 📁 Path to the output folder next to this notebook
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# 📄 Path to the HTML file
output_path = os.path.join(output_dir, "3D_KmeansKnown_layers.html")

# 💾 Save the figure
fig.write_html(output_path)
print(f"✅ Plot saved: {output_path}")


✅ Plot saved: ./output\3D_KmeansKnown_layers.html


In [7]:
# ✅ Combined 3D plot added

import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import numpy as np

# 📁 Output folder
output_dir = "./output/plots_KmeasKnown_layers"
os.makedirs(output_dir, exist_ok=True)

# 📌 Unique layers and color map
unique_layers = sorted(df_input['layer_id_2MethodKmean7'].unique())
n_layers = len(unique_layers)
colormap = cm.get_cmap('tab20', n_layers)
layer_colors_kmeans7 = {layer_id: colormap(i) for i, layer_id in enumerate(unique_layers)}

# 📐 Coordinate bounds (fixed scale)
x_min, x_max = df_input['X'].min(), df_input['X'].max()
y_min, y_max = df_input['Y'].min(), df_input['Y'].max()
z_min, z_max = df_input['Z'].min(), df_input['Z'].max()
aspect = [x_max - x_min, y_max - y_min, z_max - z_min]

# 📊 Individual 2D and 3D plots per layer
for gid in unique_layers:
    layer = df_input[df_input['layer_id_2MethodKmean7'] == gid]
    print(f'🟢 Visualizing layer {gid} ({len(layer)} points)')

    # 2D projections
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    axs[0].scatter(layer['X'], layer['Z'], s=1, color=layer_colors_kmeans7[gid])
    axs[0].set_title(f'Layer {gid} — XZ')
    axs[0].set_xlabel('X')
    axs[0].set_ylabel('Z')
    axs[0].set_xlim(x_min, x_max)
    axs[0].set_ylim(z_min, z_max)
    axs[0].grid(True)

    axs[1].scatter(layer['Y'], layer['Z'], s=1, color=layer_colors_kmeans7[gid])
    axs[1].set_title(f'Layer {gid} — YZ')
    axs[1].set_xlabel('Y')
    axs[1].set_ylabel('Z')
    axs[1].set_xlim(y_min, y_max)
    axs[1].set_ylim(z_min, z_max)
    axs[1].grid(True)

    axs[2].scatter(layer['X'], layer['Y'], s=1, color=layer_colors_kmeans7[gid])
    axs[2].set_title(f'Layer {gid} — XY')
    axs[2].set_xlabel('X')
    axs[2].set_ylabel('Y')
    axs[2].set_xlim(x_min, x_max)
    axs[2].set_ylim(y_min, y_max)
    axs[2].grid(True)

    plt.tight_layout()
    plot_2d_path = os.path.join(output_dir, f"layer_{gid}_2D.png")
    fig.savefig(plot_2d_path, dpi=300)
    plt.close(fig)

    # 3D plot for a single layer
    fig3d = plt.figure(figsize=(8, 6))
    ax = fig3d.add_subplot(111, projection='3d')
    ax.scatter(layer['X'], layer['Y'], layer['Z'], s=1, color=layer_colors_kmeans7[gid])
    ax.set_title(f'Layer {gid} — 3D')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)
    ax.set_box_aspect(aspect)
    ax.view_init(elev=30, azim=225)
    plot_3d_path = os.path.join(output_dir, f"layer_{gid}_3D.png")
    fig3d.savefig(plot_3d_path, dpi=300)
    plt.close(fig3d)

    print(f"💾 Saved: {plot_2d_path}, {plot_3d_path}")

# 🧩 Combined 3D plot for all layers
fig_all = plt.figure(figsize=(10, 8))
ax_all = fig_all.add_subplot(111, projection='3d')

for gid in unique_layers:
    layer = df_input[df_input['layer_id_2MethodKmean7'] == gid]
    ax_all.scatter(layer['X'], layer['Y'], layer['Z'], s=1, color=layer_colors_kmeans7[gid], label=f'Layer {gid}')

ax_all.set_title('3D Layered Point Cloud View (KMeans, Known N)')
ax_all.set_xlabel('X')
ax_all.set_ylabel('Y')
ax_all.set_zlabel('Z')
ax_all.set_xlim(x_min, x_max)
ax_all.set_ylim(y_min, y_max)
ax_all.set_zlim(z_min, z_max)
ax_all.set_box_aspect(aspect)
ax_all.view_init(elev=30, azim=225)

plot_all_path = os.path.join(output_dir, "all_layers_3D.png")
fig_all.savefig(plot_all_path, dpi=300)
plt.close(fig_all)

print(f"📌 Combined 3D plot saved: {plot_all_path}")


C:\Users\kopil\AppData\Local\Temp\ipykernel_41388\2375447389.py:16: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



🟢 Visualizing layer 0 (1653 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_0_2D.png, ./output/plots_KmeasKnown_layers\layer_0_3D.png
🟢 Visualizing layer 1 (3824 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_1_2D.png, ./output/plots_KmeasKnown_layers\layer_1_3D.png
🟢 Visualizing layer 2 (3804 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_2_2D.png, ./output/plots_KmeasKnown_layers\layer_2_3D.png
🟢 Visualizing layer 3 (3933 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_3_2D.png, ./output/plots_KmeasKnown_layers\layer_3_3D.png
🟢 Visualizing layer 4 (3337 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_4_2D.png, ./output/plots_KmeasKnown_layers\layer_4_3D.png
🟢 Visualizing layer 5 (3775 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_5_2D.png, ./output/plots_KmeasKnown_layers\layer_5_3D.png
🟢 Visualizing layer 6 (2639 points)
💾 Saved: ./output/plots_KmeasKnown_layers\layer_6_2D.png, ./output/plots_KmeasKnown_layers\layer_6_3D.png
📌 Comb

In [8]:
import os

output_dir = r"C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\2MethodClusterKMeansKnown\output"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "2MethodClusterKMeansKnown.csv")
df_input.to_csv(output_path, index=False)

print(f"✅ Saved: {output_path}")


✅ Saved: C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\2MethodClusterKMeansKnown\output\2MethodClusterKMeansKnown.csv
